In [10]:
import re, string,os
from glob import glob as gb
import pandas as pd
from tqdm import tqdm
from collections import Counter
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
from functions import *
import calendar
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import defaultdict

from nltk.corpus import stopwords

base_path = "/home/ruben/Documents/GitHub/ParlaMintCase"

In [5]:
df = data_loader.period(language="gb",data_version="lemmatized",start_date="2020-01",end_date="2020-12")
df = df[df['id'].astype(str).str.len() > 2].dropna().reset_index(drop=True)
df['date'] = [utils.find_date(x)[:7] for x in df['id']]
df = df[df.id.astype(str).str.contains('-commons')]
df = utils.add_metadata(df,'gb')
df['text'] = utils.preprocess_(df['text'],stopwords=set(stopwords.words('english')) | set("hon. friend right gentleman".split(' ')))
# df['text'] = utils.preprocess_(df['text'],stopwords=set(stopwords.words('dutch')) | set("wij zij het jij ons jullie".split(' ')))
df['text'] = [re.sub(r'[^A-Za-z0-9 ]+', '', str(s)) for s in df['text']]

found 289 files


In [11]:
def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()
    for text in sentences:
        # preprocessing (use tokenizer instead)
        text = text.lower().split()
        # iterate over sentences
        for i in range(len(text)):
            token = text[i]
            vocab.add(token)  # add to vocab
            next_token = text[i+1 : i+1+window_size]
            for t in next_token:
                key = tuple( sorted([t, token]) )
                d[key] += 1

    # formulate the dictionary into dataframe
    vocab = sorted(vocab) # sort vocab
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in d.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

In [7]:
dfm = df[df['date'] == '2020-01'].reset_index(drop=True)

In [12]:
df_counts = co_occurrence(list(dfm['text']),5)

In [14]:
def pmi(df, positive=True):
    col_totals = df.sum(axis=0)
    total = col_totals.sum()
    row_totals = df.sum(axis=1)
    expected = np.outer(row_totals, col_totals) / total
    df = df / expected
    # Silence distracting warnings about log(0):
    with np.errstate(divide='ignore'):
        df = np.log(df)
    df[np.isinf(df)] = 0.0  # log(0) = 0
    if positive:
        df[df < 0] = 0.0
    return df

In [ ]:
df_ppmi = pmi(df_counts,positive=True)